In [1]:
import http.server    # part of python no install
import json 
import mysql.connector
from urllib.parse import urlparse, parse_qs    # part of python no install

# pip install mysql-connector-python
# pip install simplejson

# Database connection config
db_config = {
    'user': 'root',
    'password': 'RootRoot',
    'host': 'localhost',
    'database': 'contactlist2'
}

class ContactList:
    def __init__(self):
        self.conn = mysql.connector.connect(**db_config)
        self.cursor = self.conn.cursor(dictionary=True)

    def read_contacts(self):
        self.cursor.execute("select * from contacts")
        return self.cursor.fetchall()

    def create_contact(self, fullName, phone, email):
        query = "insert into contacts(fullName, phone, email) values(%s, %s, %s)"
        self.cursor.execute(query,(fullName, phone, email))
        self.conn.commit()
        return {'id': self.cursor.lastrowid, 'fullName': fullName, 'phone': phone, 'email': email}
    
    def get_contact_by_id(self, contact_id):
        query = "select * from contacts where id = %s;"
        self.cursor.execute(query, (contact_id,))
        return self.cursor.fetchone()
    
    def update_contact(self, contact_id, new_phone=None, new_email=None):
        query = "update contacts set phone = %s, email = %s where id = %s"
        self.cursor.execute(query, (new_phone, new_email, contact_id))
        self.conn.commit()
        return self.cursor.rowcount > 0

    def delete_contact(self, contact_id):
        query = "delete from contacts where id = %s"
        self.cursor.execute(query, (contact_id,))
        self.conn.commit()
        return self.cursor.rowcount > 0
    
contact_list = ContactList()

class RequestHandler(http.server.BaseHTTPRequestHandler):

    def do_GET(self):
        parsed_path = urlparse(self.path)
        path_parts = parsed_path.path.split('/')  #localhost:8080/contacts
        if len(path_parts) == 2 and path_parts[1] == "contacts":  # Get All
            self.send_response(200)
            self.send_header("Content-type", "application/json")
            self.end_headers()
            self.wfile.write(json.dumps(contact_list.read_contacts()).encode())
        elif len(path_parts) == 3 and path_parts[1] == "contacts": # localhost:8080/contacts/123
            contact_id = path_parts[2]
            contact = contact_list.get_contact_by_id(contact_id)
            if contact:
                self.send_response(200)
                self.send_header("Content-type", "application/json")
                self.end_headers()
                self.wfile.write(json.dumps(contact).encode())
            else:
                self.send_response(404)
                self.end_headers()
        else:
            self.send_response(404)
            self.end_headers()

    def do_POST(self):
        pass

    def do_PUT(self):
        pass

    def do_DELETE(self):
        pass






def run(server_class=http.server.HTTPServer, handler_class=RequestHandler, port=8080):
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)
    print(f"Starting server on port {port}...")
    httpd.serve_forever()
    
if __name__ == "__main__":
    run()

Starting server on port 8080...


127.0.0.1 - - [05/Sep/2024 16:12:13] "GET /contacts/3 HTTP/1.1" 200 -


KeyboardInterrupt: 